In [1]:
#import pymysql
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split#,GridSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC  
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer
from sklearn import cross_validation,metrics
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, classification_report, roc_curve, auc  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
'''
conn = pymysql.connect(host='192.168.1.131',user='yichen', password='0000', database='yelp_db', charset='utf8')
cur = conn.cursor()

#query="select id as user_id, review_count,yelping_since,useful,funny,cool,fans,average_stars, compliment_hot+ compliment_more+ compliment_profile+ compliment_cute+ compliment_list+ compliment_note+ compliment_plain+compliment_cool+compliment_funny+compliment_writer+compliment_photos as sum_compliment from user"
query='select id as review_id,business_id,user_id,stars,date,useful,funny,cool from review'
cur.execute(query)
#cur.execute('commit')
t=cur.fetchall()
#user_info_50000=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])
review_info_notext=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])
print('done')
review_info_notext.to_csv('E:/DATA/DATA in CU Boulder/CSCI 5622/review_info_notext.csv')
'''

F:\Tools\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
F:\Tools\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


'\nconn = pymysql.connect(host=\'192.168.1.131\',user=\'yichen\', password=\'0000\', database=\'yelp_db\', charset=\'utf8\')\ncur = conn.cursor()\n\n#query="select id as user_id, review_count,yelping_since,useful,funny,cool,fans,average_stars, compliment_hot+ compliment_more+ compliment_profile+ compliment_cute+ compliment_list+ compliment_note+ compliment_plain+compliment_cool+compliment_funny+compliment_writer+compliment_photos as sum_compliment from user"\nquery=\'select id as review_id,business_id,user_id,stars,date,useful,funny,cool from review\'\ncur.execute(query)\n#cur.execute(\'commit\')\nt=cur.fetchall()\n#user_info_50000=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])\nreview_info_notext=pd.DataFrame(list(t), columns = [i[0] for i in cur.description])\nprint(\'done\')\nreview_info_notext.to_csv(\'E:/DATA/DATA in CU Boulder/CSCI 5622/review_info_notext.csv\')\n'

In [2]:
all_withoutTextFea=pd.read_csv('review_cat_user_userFeature.csv')
del all_withoutTextFea['Unnamed: 0']
text1=pd.read_csv('review_data.csv')
text2=pd.read_csv('data_final.csv')
user_userFea_business_50000=pd.read_csv('user_userFea_business_50000.csv')

E:\Tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
text_ori=text1[['review_id','reviews']]
text_polar_sub=text2[['review_id','reviews_polarity','reviews_subjectivity']].drop_duplicates()

In [4]:
#del user_userFea_business_50000['Unnamed: 0']
#unique_id=text2[['review_id']].drop_duplicates()
#text_polar_sub_unique=pd.merge(text_ori,text_polar_sub,how='inner',on='review_id')
#all_without_len_uc=pd.merge(text_polar_sub_unique,user_userFea_business_50000,how='inner',on='review_id')
#text_ori['review_length']=0
#for i in range(len(text_ori)):
#    text_ori['review_length'][i]=len(text_ori['reviews'][i].split())
#all_features=pd.merge(all_withoutTextFea,all_without_len_uc[['review_id','reviews','reviews_polarity','reviews_subjectivity']],on='review_id',how='inner')
#all_features.to_csv('all_features_without_tfidfBoG.csv')
#all_features_without_uc



#all_features_without_uc=pd.read_csv('all_features_without_uc_tfidfBoG.csv',encoding='gb18030')
#all_features
#del all_features['Unnamed: 0']
#del all_features_without_uc['Unnamed: 0']

#all_features_notfidfBoG
#business=all_features_without_uc[['business_id','business_stars']].drop_duplicates()
#all_features_notfidfBoG=pd.merge(all_features,business,how='inner',on='business_id')
all_features_notfidfBoG=pd.read_csv('all_features_noTFIDFBoG.csv',encoding='gb18030')
#all_without_uctfidfBoG=pd.read_csv('all_features_noUC_TFIDFBoG.csv',encoding='gb18030')

In [6]:
class OtherFeatureTransformer(BaseEstimator, TransformerMixin):  
    def __init__(self):
        self.feature_names=[]
    def fit(self, examples):
        # return self and nothing else 
        return self

    def transform(self, examples):
        #print(len(examples))
        from scipy.sparse import csr_matrix
        self.X = np.zeros((len(examples[0]), 20))
        
        for j in range(20):
            #for i in range(len(examples[0])):
            self.X[:,j] = examples[j]
        #print(self.X.shape)
        return csr_matrix(self.X)
    def get_feature_names(self):
        return self.feature_names 
class Feat:
    def __init__(self):
        pass
    def build_train_features(self, examples):
        self.tfidf= TfidfVectorizer(stop_words = 'english',analyzer='word',ngram_range=(1,4))
        self.count=CountVectorizer()
        self.other=OtherFeatureTransformer()
        self.vectorizer = FeatureUnion( 
        [       
             #word tfidf
            ('tfidf', 
              Pipeline([('extract_field', FunctionTransformer(lambda x: x[0], validate = False)),
                        ('count',self.tfidf)])),            
             #appearance of some tropes
            ('bagofwords', 
              Pipeline([('extract_field', FunctionTransformer(lambda x: x[0], validate = False)),
                        ('count', self.count)])),
             #type of sentiment
            ('other features', 
            Pipeline([('extract_field', FunctionTransformer(lambda x: [x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],
                                                                       x[12],x[13],x[14],x[15],x[16],x[17],x[18],x[19],x[20],],
                                                            validate = False)),
                        ('other', self.other)]))
            #
            #('other features', 
            # Pipeline([('extract_field', FunctionTransformer(lambda x: [x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13],x[14]], validate = False)),
            #            ('other', self.other)]))
    
        ])
        return self.vectorizer.fit_transform(examples)
    def build_test_features(self,examples):
        return self.vectorizer.transform(examples)

In [7]:
train, test = train_test_split(all_features_notfidfBoG, test_size=0.2)
train_fea=train
test_fea=test
train_lab=[]
test_lab=[]
ini=0
for index, row in train.iterrows():
    if row['review_stars'] ==5:
        train_lab.append(5)
    elif (row['review_stars'] ==4):
        train_lab.append(4)
    elif row['review_stars'] == 3:
        train_lab.append(3)
    elif row['review_stars'] ==2:
        train_lab.append(2)
    else:
        train_lab.append(1)
    ini += 1
print('training labels generated')
ini = 0
for index, row in test.iterrows():
    if row['review_stars'] ==5:
        test_lab.append(5)
    elif (row['review_stars'] ==4):
        test_lab.append(4)
    elif row['review_stars'] == 3:
        test_lab.append(3)
    elif row['review_stars'] ==2:
        test_lab.append(2)
    else:
        test_lab.append(1)
    ini += 1 
print('testing labels generated')

training labels generated
testing labels generated


In [8]:
getFeat=Feat()
#train_feature=getFeat.build_train_features([list(train_fea['user_review_count']),list(train_fea['user_useful']),list(train_fea['user_funny']),list(train_fea['user_cool']),list(train_fea['user_fans']),list(train_fea['user_average_stars']),list(train_fea['user_sum_compliment'])])
#test_feature=getFeat.build_test_features([list(test_fea['user_review_count']),list(test_fea['user_useful']),list(test_fea['user_funny']),list(test_fea['user_cool']),list(test_fea['user_fans']),list(test_fea['user_average_stars']),list(test_fea['user_sum_compliment'])])
train_feature=getFeat.build_train_features([list(train_fea['reviews']),list(train_fea['reviews_polarity']),
                                            list(train_fea['reviews_subjectivity']),list(train_fea['business_stars']),
                                            list(train_fea['review_stars']),list(train_fea['review_useful']),list(train_fea['review_funny']),
                                            list(train_fea['review_cool']),list(train_fea['user_review_count']),
                                            list(train_fea['user_useful']),list(train_fea['user_funny']),list(train_fea['user_cool']),
                                            list(train_fea['user_fans']),list(train_fea['user_average_stars']),
                                            list(train_fea['user_sum_compliment']),list(train_fea['uc_review_count']),
                                            list(train_fea['uc_avg_stars']),list(train_fea['uc_avg_review_useful']),
                                            list(train_fea['uc_avg_review_funny']),list(train_fea['uc_avg_review_cool']),
                                            list(train_fea['business_stars'])
])
test_feature=getFeat.build_test_features([list(test_fea['reviews']),list(test_fea['reviews_polarity']),
                                          list(test_fea['reviews_subjectivity']),list(test_fea['business_stars']),
                                          list(test_fea['review_stars']),list(test_fea['review_useful']),list(test_fea['review_funny']),
                                          list(test_fea['review_cool']),list(test_fea['user_review_count']),
                                          list(test_fea['user_useful']),list(test_fea['user_funny']),list(test_fea['user_cool']),
                                          list(test_fea['user_fans']),list(test_fea['user_average_stars']),
                                          list(test_fea['user_sum_compliment']),list(test_fea['uc_review_count']),
                                          list(test_fea['uc_avg_stars']),list(test_fea['uc_avg_review_useful']),
                                          list(test_fea['uc_avg_review_funny']),list(test_fea['uc_avg_review_cool']),
                                          list(test_fea['business_stars'])])

In [43]:
#all_features_notfidfBoG

In [9]:
### scale the data first
#scaler = preprocessing.StandardScaler().fit(train_feature)
#scaler.transform(train_feature)
#scaler.transform(test_feature)
train_feature=Normalizer().fit_transform(train_feature)
test_feature=Normalizer().fit_transform(test_feature)

In [11]:
#logreg = linear_model.LogisticRegression(C=10.0)
#logreg.fit(train_feature, train_lab)

#pred_logreg = logreg.predict(test_feature)
#acc_logreg = logreg.score(test_rev_lst,test_lab_lst)
#pred_proba_logreg=logreg.predict_proba(test_feature)
print('ACC:',precision_score(test_lab,pred_logreg,average='micro'))  

#print('AUC:',metrics.roc_auc_score(test_lab,pred_proba_logreg[:,1]))#验证集上的auc值
#print('F1:',metrics.f1_score(test_lab,pred_logreg))
#print('Recall:',metrics.recall_score(test_lab,pred_logreg))
#print('the auc of logistic regression is %f' %test_auc)
print(metrics.confusion_matrix(test_lab,pred_logreg))#验证集上的混淆矩阵
print('RMSE:',math.sqrt(metrics.mean_squared_error(test_lab,pred_logreg)))

ACC: 0.7744354063993907
[[21743    51    42   932   215]
 [  894  6110  1829  5597   323]
 [    8   841  5324 13971   829]
 [    3    99   710 31159  9078]
 [    2     3    39  4216 71905]]
RMSE: 0.6435823623997685


<703688x22447295 sparse matrix of type '<class 'numpy.float64'>'
	with 197963052 stored elements in Compressed Sparse Row format>